In [17]:
#导入库并产生数据
import math
import numpy as np
import torch
from torch.utils import data
import random

#定义正态分布
def normal(x,mu,sigma):
    p=1/math.sqrt(2*math.pi*sigma*sigma)
    return p*np.exp(-0.5/sigma**2 *(x-mu)**2)

#制造数据集 y=Xw+b+噪声
def synthetic_data(w, b, num_examples): 
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 100)

In [18]:
#"""构造⼀个PyTorch数据迭代器"""
def load_array(data_arrays, batch_size, is_train=True): 
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)
batch_size = 10
data_iter = load_array((features, labels), batch_size)
print(next(iter(data_iter)))

[tensor([[ 1.6772,  1.1961],
        [ 0.8665,  0.0117],
        [ 1.3800,  1.0251],
        [-2.1406, -0.6759],
        [ 0.3587, -0.3520],
        [-1.1786, -0.3054],
        [ 1.9313, -0.0846],
        [ 0.8176,  0.7768],
        [ 0.4668, -1.4426],
        [-0.8177,  1.5955]]), tensor([[ 3.4865],
        [ 5.8836],
        [ 3.4613],
        [ 2.2056],
        [ 6.1055],
        [ 2.8730],
        [ 8.3308],
        [ 3.1924],
        [10.0350],
        [-2.8484]])]


In [19]:
#建立模型
from torch import nn
#nn.linear第一个参数为输入的特征，第二个为输出量
net=nn.Sequential(nn.Linear(2,1))
#初始化模型参数
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)
#定义损失函数
loss = nn.MSELoss()
#定义优化器
sgd=torch.optim.SGD(net.parameters(), lr=0.03)


In [20]:
num_epochs = 10
for epoch in range(num_epochs):
    for X,y in data_iter:
        l=loss(net(X),y)
        sgd.zero_grad()
        l.backward()
        sgd.step()
    l=loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {float(l):f}')

print([true_w,true_b])
print([net[0].weight.data,net[0].bias.data])

epoch 1, loss 9.775008
epoch 2, loss 3.132302
epoch 3, loss 1.011569
epoch 4, loss 0.325415
epoch 5, loss 0.105293
epoch 6, loss 0.034051
epoch 7, loss 0.011094
epoch 8, loss 0.003663
epoch 9, loss 0.001266
epoch 10, loss 0.000500
[tensor([ 2.0000, -3.4000]), 4.2]
[tensor([[ 1.9922, -3.3852]]), tensor([4.1860])]


In [21]:
#建立模型
from torch import nn
#nn.linear第一个参数为输入的特征，第二个为输出量
net=nn.Sequential(nn.Linear(2,1))
#初始化模型参数
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)
#定义损失函数
loss = nn.SmoothL1Loss()
#定义优化器
sgd=torch.optim.SGD(net.parameters(), lr=0.03)


In [22]:
num_epochs = 10
for epoch in range(num_epochs):
    for X,y in data_iter:
        l=loss(net(X),y)
        sgd.zero_grad()
        l.backward()
        sgd.step()
    l=loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {float(l):f}')

print([true_w,true_b])
print([net[0].weight.data,net[0].bias.data])

epoch 1, loss 3.798919
epoch 2, loss 3.609834
epoch 3, loss 3.422261
epoch 4, loss 3.236351
epoch 5, loss 3.052205
epoch 6, loss 2.869768
epoch 7, loss 2.689223
epoch 8, loss 2.510486
epoch 9, loss 2.333822
epoch 10, loss 2.159138
[tensor([ 2.0000, -3.4000]), 4.2]
[tensor([[ 0.7469, -1.0603]]), tensor([1.9512])]
